In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import os
import git
import sqlalchemy
import pyodbc 

In [2]:
# CONSTANTS
SOURCE_URL = 'https://github.com/CSSEGISandData/COVID-19'
DATA_PATH = '../data'
DAILY_REPORTS_PATH = f"{DATA_PATH}/csse_covid_19_data/csse_covid_19_daily_reports"
DAILY_REPORTS_TABLE_NAME = 'covid_daily_reports'
DAILY_REPORTS_US_TABLE_NAME = 'covid_daily_reports_us'
DAILY_REPORTS_US_PATH = f"{DATA_PATH}/csse_covid_19_data/csse_covid_19_daily_reports_us"
DAILY_REPORTS_COLUMN_NAMES = ['FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Last_Update',
       'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'Combined_Key', 'Incident_Rate', 'Case_Fatality_Ratio']
DAILY_REPORTS_US_COLUMN_NAMES = ['Province_State', 'Country_Region', 'Last_Update', 'Lat', 'Long_', 
       'Confirmed', 'Deaths', 'Recovered', 'Active', 'FIPS', 'Incident_Rate', 'Total_Test_Results',
       'People_Hospitalized', 'Case_Fatality_Ratio', 'UID', 'ISO3', 'Testing_Rate', 'Hospitalization_Rate']


Настройки подключения к БД

In [3]:
# DB CONFIG
SERVER_NAME = 'HOME-PC'
DB_NAME = 'covid'

Клонируем репозиторий с данными для этого нужно импортировать git.
Если гита нет то установить pip install gitpython

In [4]:
# Git clone
if not(os.path.exists(DATA_PATH)) or (not os.listdir(DATA_PATH)):
    git.Repo.clone_from(SOURCE_URL, DATA_PATH)

Настраиваем соединение с базой данны. Так как используем mssql нужен специальный драйвер для настройки соединения.
Также sqlalchemy поддерживает работу с pyodbc для более простой записи данных.

In [4]:
# DB connect
from sqlalchemy.engine import URL
connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER_NAME+';DATABASE='+DB_NAME+';Trusted_Connection=yes;'
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = sqlalchemy.create_engine(connection_url)

Далее описана функция для записи данных в таблицу.

In [54]:
def write_data(table_name, column_names, csv_folder_path, engine):
    for filename in os.listdir(csv_folder_path):
        if filename.endswith('.csv'):
            df = pd.read_csv(f"{csv_folder_path}/{filename}", header=None, skiprows=1)
            try:
                df.columns = column_names
                df.to_sql(
                    name=table_name,
                    con=engine,
                    index=False,
                    if_exists='append'
                )
            except:
                continue


Записываем данные в две разные таблицы

In [55]:
write_data(DAILY_REPORTS_TABLE_NAME, DAILY_REPORTS_COLUMN_NAMES, DAILY_REPORTS_PATH, engine)
write_data(DAILY_REPORTS_US_TABLE_NAME, DAILY_REPORTS_US_COLUMN_NAMES, DAILY_REPORTS_US_PATH, engine)

In [98]:
covidDfUs = pd.read_sql('covid_daily_reports_us', engine)
covidDf = pd.read_sql('covid_daily_reports', engine)

In [114]:
# Информация по пустым значениям
print((covidDfUs.isnull()).sum())

Province_State              0
Country_Region              0
Last_Update                 0
Lat                      1090
Long_                    1090
Confirmed                   0
Deaths                      0
Recovered                   0
Active                      0
FIPS                        0
Incident_Rate            1090
Total_Test_Results       1108
People_Hospitalized     26467
Case_Fatality_Ratio       531
UID                         0
ISO3                        0
Testing_Rate             1108
Hospitalization_Rate    26467
dtype: int64


In [115]:
# Информация по пустым значениям
print((covidDf.isnull()).sum())

FIPS                   360647
Admin2                 358208
Province_State          85874
Country_Region              0
Last_Update                 0
Lat                     40280
Long_                   40280
Confirmed                   0
Deaths                      0
Recovered                   0
Active                      0
Combined_Key                0
Incident_Rate           40898
Case_Fatality_Ratio     21225
dtype: int64


In [118]:
covidDfUs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31594 entries, 0 to 31627
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Province_State        31594 non-null  string        
 1   Country_Region        31594 non-null  string        
 2   Last_Update           31594 non-null  datetime64[ns]
 3   Lat                   30504 non-null  float64       
 4   Long_                 30504 non-null  float64       
 5   Confirmed             31594 non-null  int64         
 6   Deaths                31594 non-null  int64         
 7   Recovered             31594 non-null  int64         
 8   Active                31594 non-null  int64         
 9   FIPS                  31594 non-null  float64       
 10  Incident_Rate         30504 non-null  float64       
 11  Total_Test_Results    30486 non-null  float64       
 12  People_Hospitalized   5127 non-null   float64       
 13  Case_Fatality_Ra

In [117]:
covidDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1978865 entries, 0 to 1982396
Data columns (total 14 columns):
 #   Column               Dtype         
---  ------               -----         
 0   FIPS                 float64       
 1   Admin2               string        
 2   Province_State       string        
 3   Country_Region       string        
 4   Last_Update          datetime64[ns]
 5   Lat                  float64       
 6   Long_                float64       
 7   Confirmed            int64         
 8   Deaths               int64         
 9   Recovered            int64         
 10  Active               int64         
 11  Combined_Key         string        
 12  Incident_Rate        float64       
 13  Case_Fatality_Ratio  float64       
dtypes: datetime64[ns](1), float64(5), int64(4), string(4)
memory usage: 226.5 MB


In [120]:
def ChangeColumnType(column, typeName, df):
    df[column] = df[column].astype(typeName)

def DropLinesLessThanZero(df, columnName):
    return df.drop(df.index[df[columnName] < 0]) 

def FillMeanValues(df, column):
    df[column] = df[column].fillna(df[column].mean())
    return df[column]

In [104]:
for column in covidDfUs.columns:
    if (covidDfUs[column].dtype == 'object'):
        ChangeColumnType(column, 'string', covidDfUs)
        
for column in covidDf.columns:
    if (covidDf[column].dtype == 'object'):
        ChangeColumnType(column, 'string', covidDf)

In [106]:
# Удаляем строки с неопределенной датой.
covidDfUs = covidDfUs.dropna(subset=['Last_Update'])

In [107]:
print(f"covidDf: \n {covidDf['Last_Update'].describe()}")
print(f"\ncovidDfUs: \n {covidDfUs['Last_Update'].describe()}")

covidDf: 
 count                 1982397
unique                    577
top       2020-11-29 05:25:55
freq                     3995
first     2020-05-30 02:32:48
last      2021-10-11 04:21:31
Name: Last_Update, dtype: object

covidDfUs: 
 count                   31609
unique                    713
top       2020-04-25 06:32:46
freq                       59
first     2020-04-12 23:18:15
last      2021-10-11 04:31:18
Name: Last_Update, dtype: object


C:\Temp/ipykernel_20108/3082267031.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(f"covidDf: \n {covidDf['Last_Update'].describe()}")
C:\Temp/ipykernel_20108/3082267031.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(f"\ncovidDfUs: \n {covidDfUs['Last_Update'].describe()}")


In [109]:
covidDf = DropLinesLessThanZero(covidDf, 'Deaths')
covidDfUs = DropLinesLessThanZero(covidDfUs, 'Deaths')
covidDf = DropLinesLessThanZero(covidDf, 'Recovered')
covidDfUs = DropLinesLessThanZero(covidDfUs, 'Active')
covidDfUs = DropLinesLessThanZero(covidDfUs, 'Recovered')
covidDf = DropLinesLessThanZero(covidDf, 'Active')


In [113]:
covidDf['Deaths'] = FillMeanValues(covidDf, 'Deaths')
covidDf['Recovered'] = FillMeanValues(covidDf, 'Recovered')
covidDf['Active'] = FillMeanValues(covidDf, 'Active')
covidDfUs['Deaths'] = FillMeanValues(covidDfUs, 'Deaths')
covidDfUs['Recovered'] = FillMeanValues(covidDfUs, 'Recovered')
covidDfUs['Active'] = FillMeanValues(covidDfUs, 'Active')

In [119]:
ChangeColumnType('Deaths', 'int64', covidDf)
ChangeColumnType('Deaths', 'int64', covidDfUs)
ChangeColumnType('Recovered', 'int64', covidDf)
ChangeColumnType('Recovered', 'int64', covidDfUs)
ChangeColumnType('Active', 'int64', covidDfUs)
ChangeColumnType('Active', 'int64', covidDf)